# Movie Analysis: Data Scrubbing

## About:
In the data scrubbing phase I will focus on cleaning up the columns I plan on using, and building up the data frame I will use for the EDA phase:

1. US Gross Revenue
2. Genre
3. Actors
4. Time of Year (date)
5. Keywords (content)

### Project imports:

In [1]:
# imports for entire data gathering phase
import pandas as pd 
import os

## 1. US Gross Revene
This column will be how we measure the other columns, so we will start here and drop any rows that don't have this information.

In [2]:
revenue_path = os.path.join(os.pardir, 'data', 'interim', 'money.csv')
revenue_df = pd.read_csv(revenue_path)

In [3]:
revenue_df.head()

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross
0,tt2488496,Star Wars: Episode VII - The Force Awakens,2015,J.J. Abrams,Lucasfilm,[US],519,$245MM,$937MM
1,tt4154796,Avengers: Endgame,2019,Anthony Russo,Marvel Studios,[US],111,$356MM,$858MM
2,tt0499549,Avatar,2009,James Cameron,Twentieth Century Fox,[US],533,$237MM,$761MM
3,tt1825683,Black Panther,2018,Ryan Coogler,Marvel Studios,[US],269,$200MM,$700MM
4,tt4154756,Avengers: Infinity War,2018,Anthony Russo,Marvel Studios,[US],376,$321MM,$679MM


In [4]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14702 entries, 0 to 14701
Data columns (total 9 columns):
imdb_id          14700 non-null object
title            14700 non-null object
year             14685 non-null object
director         14671 non-null object
production_co    14345 non-null object
region_code      13447 non-null object
rank             14688 non-null object
budget_usd       7996 non-null object
us_gross         14598 non-null object
dtypes: object(9)
memory usage: 1.0+ MB


### Changes:

1. Convert 'us_gross', and 'budget_usd' values into floats. That means stripping the non-number characters out as well as changing 'MM' to ',000,000'.
2. Convert year column to int, so the years don't have the trailing .0.
3. region_code does not need the brackets around the abbreviations.

In [5]:
# Created 3/22/2020 with current exhange values. Values not adjusted for the date the movie was created.
def get_conversion_rate(value):
    """Get exchange rate for given currency code
    
    Arguments:
        value (string): String with currency code or symbol in it

    Returns:
        rate (float): Conversion rate to usd
    """
    if '£' in value:
        return 0.854
    elif '€' in value:
        return 0.9334
    elif 'AUD' in value:
        return 1.7229
    elif 'CAD' in value:
        return 1.435
    elif 'FRF' in value:
        return 6.55957 * 0.9334
    elif 'INR' in value:
        return 75.394
    elif 'THB' in value:
        return 32.68
    elif 'EM' in value:
        return 0 # cant find info on EM
    elif 'JPY' in value:
        return 110.75
    elif 'SKW' in value:
        return 1254.45
    elif 'HUF' in value:
        return 327.94
    elif 'NGN' in value:
        return 364
    elif 'CNY' in value:
        return 7.0950
    elif 'ESP' in value:
        return 155.42826
    elif 'RUR' in value:
        return 79.87
    elif 'HKD' in value:
        return 7.7570
    elif 'ISK' in value:
        return 140.490
    elif 'PHP' in value:
        return 51.19
    elif 'DKK' in value:
        return 6.9716
    elif 'CZK' in value:
        return 25.5620
    elif 'SKK' in value:
        return 10.3753
    elif 'NOK' in value:
        return 11.7890
    elif 'MXN' in value:
        return 24.4215
    elif 'JMD' in value:
        return 135.07
    elif 'PLN' in value:
        return 4.23
    else:
        return 1

In [6]:
def strip_currency_code(value):
    """Strips currency code from front of currency string

    Arguments: 
        value (string): currency amount prefaced with currency code
    
    Returns:
        value (string): value without the currency code
    """
    if value[:1] in '$£€':
        return value[1:]
    else:
        return value[3:]

In [7]:
def convert_money(value):
    """Takes currency string and parses it into correct amount in USD
    
    Arguments:
        value (string): currency in form: CAD 345.3B 

    Returns:
        value (int): currency converted to USD and in standard numeric form
    """
    # type checking
    if (type(value) != str):
        return None
    # check currency sign and get coefficient
    coef = get_conversion_rate(value)
    value = strip_currency_code(value)
    if 'K' in value:
        value = float(value.strip('K')) * 1000 * coef
    elif 'MM' in value:
        value = float(value.strip('MM')) * 1000000 * coef
    elif 'B' in value:
        value = float(value.strip('B')) * 1000000000 * coef
    else:
        value = float(value.strip()) * coef
    return int(value)

In [8]:
revenue_df['us_gross'] = revenue_df['us_gross'].apply(convert_money)

In [9]:
revenue_df['budget_usd'] = revenue_df['budget_usd'].apply(convert_money)

In [10]:
revenue_df['budget_usd'].isna().sum()

6706

In [11]:
revenue_df.sample(5)

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross
62,tt1477834,Aquaman,2018,James Wan,DC Comics,[US],516,160000000.0,335000000.0
4722,tt1604171,Prom,2011,Joe Nussbaum,Rickshaw Productions,[US],"14,200",8000000.0,10000000.0
1141,tt0241303,Chocolat,2000,Lasse Hallström,Miramax,[US],"2,869",25000000.0,72000000.0
4915,tt0891592,Street Fighter: The Legend of Chun-Li,2009,Andrzej Bartkowiak,Hyde Park Entertainment,[US],"11,405",18000000.0,8700000.0
9229,tt1213672,Billy: The Early Years,2008,Robby Benson,Solex Productions,[GB],"58,196",NaN,347000.0


Now for region code. We actually don't need this column so we will drop it.

In [12]:
revenue_df.drop(columns='region_code', inplace=True)

For the 'year' column we went ahead and dropped the missing rows, because there were only 6 of them.

In [13]:
revenue_df.isna().sum()

imdb_id             2
title               2
year               17
director           31
production_co     357
rank               14
budget_usd       6706
us_gross          104
dtype: int64

Cleaning up Nan values:

In [14]:
# first, change the missing values from budget to -1, so we dont drop 1910 rows.
revenue_df['budget_usd'] = revenue_df['budget_usd'].fillna(-1)

In [15]:
# also, fill in the production_co missing values with an 'Unknown'
revenue_df['production_co'] = revenue_df['production_co'].fillna('Unknown')

In [16]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14702 entries, 0 to 14701
Data columns (total 8 columns):
imdb_id          14700 non-null object
title            14700 non-null object
year             14685 non-null object
director         14671 non-null object
production_co    14702 non-null object
rank             14688 non-null object
budget_usd       14702 non-null float64
us_gross         14598 non-null float64
dtypes: float64(2), object(6)
memory usage: 919.0+ KB


In [17]:
revenue_df = revenue_df.dropna()

In [18]:
revenue_df.sample(5)

,imdb_id,title,year,director,production_co,rank,budget_usd,us_gross
12499,tt2386502,A Long Way Off,2014,Michael Davis,Prodigafilm,"133,046",500000.0,30000.0
9345,tt9205986,You Are Here: A Come From Away Story,2018,Moze Mossanen,MDF Productions,"25,016",-1.0,319000.0
12604,tt0079820,The King and the Mockingbird,1980,Paul Grimault,Les Films Paul Grimault,"49,936",-1.0,28000.0
5596,tt3385524,Stan & Ollie,2018,Jon S. Baird,Entertainment One,"4,085",10000000.0,5500000.0
1092,tt0164334,Along Came a Spider,2001,Lee Tamahori,Paramount Pictures,"2,962",60000000.0,74000000.0


Now for dropping duplicates:

In [19]:
revenue_df = revenue_df.drop_duplicates()

In [20]:
revenue_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14431 entries, 0 to 14598
Data columns (total 8 columns):
imdb_id          14431 non-null object
title            14431 non-null object
year             14431 non-null object
director         14431 non-null object
production_co    14431 non-null object
rank             14431 non-null object
budget_usd       14431 non-null float64
us_gross         14431 non-null float64
dtypes: float64(2), object(6)
memory usage: 1014.7+ KB


In [21]:
revenue_df.sample(3)

,imdb_id,title,year,director,production_co,rank,budget_usd,us_gross
8178,tt7147540,Made in India,2018,Sharat Katariya,Yash Raj Films,"33,210",6200000.0,791000.0
9207,tt1599975,The Princess of Montpensier,2010,Bertrand Tavernier,Paradis Films,"25,385",-1.0,352000.0
12918,tt1756799,The Day,2011,Douglas Aarniokoski,Content Media Corporation International,"17,493",-1.0,21000.0


### Save as CSV

In [22]:
revenue_save_path = os.path.join(os.pardir, 'data', 'processed', 'revenue.csv')
revenue_df.to_csv(revenue_save_path, index=False)

In [23]:
test_revenue_save = pd.read_csv(revenue_save_path)
test_revenue_save.sample(3)

,imdb_id,title,year,director,production_co,rank,budget_usd,us_gross
1830,tt1872194,The Judge,2014,David Dobkin,Warner Bros.,"2,631",50000000.0,47000000.0
11625,tt0843850,Finishing the Game: The Search for a New Bruce...,2007,Justin Lin,Barnstorm Pictures,"65,097",500000.0,53000.0
13878,tt5770864,The Man from Earth: Holocene,2017,Richard Schenkman,Falling Sky Entertainment,"13,356",-1.0,5000.0


## 2. Genre:
For genre we will need a dataset that lists each movie and it's genre. To analyze the success of the genre, we will need to examine the relationship of genre to the revenue earned.

Bringing in the list of movie titles:

In [24]:
titles_path = os.path.join(os.pardir, 'data', 'raw', 'movies.csv')

In [25]:
genres_df = pd.read_csv(titles_path)
genres_df.head()

,tconst,primaryTitle,startYear,genres
0,tt0000009,Miss Jerry,1894,Romance
1,tt0000147,The Corbett-Fitzsimmons Fight,1897,"Documentary,News,Sport"
2,tt0000335,Soldiers of the Cross,1900,"Biography,Drama"
3,tt0000502,Bohemios,1905,\N
4,tt0000574,The Story of the Kelly Gang,1906,"Biography,Crime,Drama"


In [26]:
genres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545821 entries, 0 to 545820
Data columns (total 4 columns):
tconst          545821 non-null object
primaryTitle    545821 non-null object
startYear       545821 non-null object
genres          545821 non-null object
dtypes: object(4)
memory usage: 16.7+ MB


### Changes:
Looking at the initial dataframe, these are the things I would like to change:
1. Change column names
2. Drop original_title and runtime_minutes columns

In [27]:
genres_df = genres_df.rename(columns={'tconst': 'imdb_id', 'primaryTitle': 'title', 'startYear': 'year'})

In [28]:
genres_df.sample(3)

,imdb_id,title,year,genres
56304,tt0077527,Exit 7,1978,"Crime,Drama"
104260,tt0165656,Brutes and Savages,1978,Documentary
195312,tt0396177,Poprask na silnici E 4,1980,Comedy


That looks good. Let me deal with Nan's:

In [29]:
genres_df.isna().sum()

imdb_id    0
title      0
year       0
genres     0
dtype: int64

### Save as CSV

In [30]:
genres_save_path = os.path.join(os.pardir, 'data', 'processed', 'genres.csv')
genres_df.to_csv(genres_save_path, index=False)

In [31]:
test_genres_save = pd.read_csv(genres_save_path)
test_genres_save.sample(3)

,imdb_id,title,year,genres
290435,tt1427220,Heart Land,2012,"Comedy,Romance"
192878,tt0387460,The Off Season,2004,Horror
348912,tt2381935,FedCon XVI: Drone Heart,2007,Documentary


## 3. Actors
These columns will be key in identifying the people who have the ability to produce high quality work on a consistent basis.

In [32]:
people_path = os.path.join(os.pardir, 'data', 'raw', 'imdb.name.basics.csv')
people_df = pd.read_csv(people_path)

In [33]:
people_df.sample(3)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
377031,nm7499551,Michael P. Nagy,NaN,NaN,"art_department,production_designer,actor","tt5235954,tt4288728,tt7164814,tt4711924"
515374,nm7566094,Raviteja Mudigere,NaN,NaN,"director,writer",tt5000210
599656,nm8718962,Zhang Zikun,NaN,NaN,actor,tt6437856


### Changes:
Some cleanup tasks:
1. Change name of primary_name column to 'name'
2. Select all the actors and actress
3. Drop birth_year, death_year, known_for_titles

In [34]:
people_df.sample(3)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
245458,nm4022004,Abderrazak Zitouny,NaN,NaN,"actor,director","tt3592504,tt1706341,tt5237736"
596947,nm8919733,Serhad Öztürk,NaN,NaN,producer,"tt6768850,tt7711182"
397523,nm7006672,Adhiraj Ganguly,NaN,NaN,actor,"tt4658786,tt4300302,tt8489884"


In [35]:
people_df = people_df.rename(columns={'primary_name': 'name'})

In [36]:
def can_act(professions):
    if type(professions) != str:
        return False
    if 'actor' in professions or 'actress' in professions:
        return True
    else:
        return False

In [37]:
people_df['can_act'] = people_df['primary_profession'].apply(can_act)

In [38]:
people_df.sample(3)

,nconst,name,birth_year,death_year,primary_profession,known_for_titles,can_act
314673,nm4855396,Péter Gerõcs Asia Dér,NaN,NaN,director,tt2185475,False
601511,nm9624421,Jared Nelson,NaN,NaN,actor,"tt8549254,tt7099566,tt9185410,tt8585940",True
564985,nm9800839,Thalapathy Prabhu,NaN,NaN,director,NaN,False


Okay, we will grab all the actors and directors and make individual dataframes for them:

In [39]:
actors_df = people_df[people_df['can_act'] == True]

And now we can drop the unwanted columns:

In [40]:
drop_columns = ['primary_profession', 'can_act', 'birth_year', 'death_year', 'known_for_titles']
actors_df = actors_df.drop(columns=drop_columns)

In [41]:
actors_df.sample(3)

,nconst,name
538191,nm7410053,Ashley Brasel
3843,nm0091480,Michael Bofshever
393149,nm5063278,Francesca Testasecca


Let's check for missing values:

In [42]:
actors_df.isna().sum()

nconst    0
name      0
dtype: int64

There we go. A very large list of actors and actresses. We can join them to the titles and see if there are any patterns amongst the top performing titles.

### Save as CSV

In [43]:
actors_save_path = os.path.join(os.pardir, 'data', 'processed', 'actors.csv')
actors_df.to_csv(actors_save_path, index=False)

In [44]:
test_actors_save = pd.read_csv(actors_save_path)
test_actors_save.sample(3)

,nconst,name
3771,nm0022963,Jana Altmanová
254049,nm8747452,Yannick Hornecker
227527,nm5930666,Aruna Balaraj


## 4. Time of Year (date)
Time of year will be an important metric to discover the most opportune time to release a film.

In [45]:
date_path = os.path.join(os.pardir, 'data', 'raw', 'tmdb_movies.csv')
date_df = pd.read_csv(date_path)

In [46]:
date_df.sample(3)

,imdbId,budget,revenue,originalTitle,releaseDate
26385,tt1772424,21000000,6700000.0,Et maintenant on va où ?,2011-05-16
15822,tt0113965,6400000,6.0,Never Talk to Strangers,1995-10-20
31878,tt0110857,0,126247.0,Police Academy: Mission to Moscow,1994-06-09


### Changes:
We only need a couple columns from this set:
1. title
2. release_date

The column names are ok as well, so this will be very simple.

In [47]:
date_df = date_df.drop_duplicates()

In [48]:
date_df = date_df.rename(columns={'imdbId': 'imdb_id', 'originalTitle': 'title', 'releaseDate': 'date'})

In [49]:
date_df = date_df[['imdb_id', 'title', 'date']]

In [50]:
date_df = date_df.dropna()

In [51]:
date_df.sample(3)

,imdb_id,title,date
16756,tt0089877,Rainbow Brite and the Star Stealer,1985-11-15
42770,tt2259360,搜索,2012-07-06
10893,tt0087231,The Falcon and the Snowman,1985-01-25


In [52]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14466 entries, 0 to 43255
Data columns (total 3 columns):
imdb_id    14466 non-null object
title      14466 non-null object
date       14466 non-null object
dtypes: object(3)
memory usage: 452.1+ KB


In [53]:
date_df.isna().sum()

imdb_id    0
title      0
date       0
dtype: int64

### Save to CSV

In [54]:
date_save_path = os.path.join(os.pardir, 'data', 'processed', 'date.csv')
date_df.to_csv(date_save_path, index=False)

In [55]:
test_date_save = pd.read_csv(date_save_path)
test_date_save.sample(3)

,imdb_id,title,date
13948,tt3122842,The Automatic Hate,2015-03-15
12898,tt0208911,The King Is Alive,2000-05-11
8306,tt0134154,Ride with the Devil,1999-11-24


## 5. Keywords (content)

In [56]:
keywords_path = os.path.join(os.pardir, 'data', 'raw', 'tmdb_keywords.csv')
keywords_df = pd.read_csv(keywords_path)

In [57]:
keywords_df.sample(3)

,imdbId,keywordId,keyword
190425,tt0283897,15087,iraq war
59010,tt4034228,9253,slapstick
140106,tt0091814,9974,obesity


In [58]:
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269939 entries, 0 to 269938
Data columns (total 3 columns):
imdbId       269939 non-null object
keywordId    269939 non-null int64
keyword      269939 non-null object
dtypes: int64(1), object(2)
memory usage: 6.2+ MB


This is a simple dataframe, when I created it I knew exactly the columns I would use. 

I do need to change the column names from camelCase to snake_case (node.js uses camelCase):

In [59]:
keywords_df = keywords_df.rename(columns={'imdbId': 'imdb_id', 'keywordId': 'keyword_id'})

In [60]:
keywords_df.sample(3)

,imdb_id,keyword_id,keyword
107644,tt0452702,577,black people
29652,tt0373051,9878,fighter jet
209117,tt0261983,33451,masturbation


In [61]:
keywords_df.isna().sum()

imdb_id       0
keyword_id    0
keyword       0
dtype: int64

### Save to CSV

In [62]:
keywords_save_path = os.path.join(os.pardir, 'data', 'processed', 'keywords.csv')
keywords_df.to_csv(keywords_save_path, index=False)

In [63]:
test_keywords_save = pd.read_csv(keywords_save_path)
test_keywords_save.sample(3)

,imdb_id,keyword_id,keyword
243584,tt1922645,6091,war
244819,tt1276996,193650,pickles
46969,tt7401588,1919,mayor


## Building Dataset
In this section I will combine all the individual datasets into one large dataframe that I can explore in the EDA phase.

In [64]:
# joining revenue with genres:
combined_df = revenue_df.set_index('imdb_id').join(genres_df.set_index('imdb_id'), rsuffix='_rev')

In [66]:
combined_df.head(3)

,title,year,director,production_co,rank,budget_usd,us_gross,title_rev,year_rev,genres
imdb_id,,,,,,,,,,
tt2488496,Star Wars: Episode VII - The Force Awakens,2015,J.J. Abrams,Lucasfilm,519,245000000.0,937000000.0,Star Wars: Episode VII - The Force Awakens,2015,"Action,Adventure,Sci-Fi"
tt4154796,Avengers: Endgame,2019,Anthony Russo,Marvel Studios,111,356000000.0,858000000.0,Avengers: Endgame,2019,"Action,Adventure,Drama"
tt0499549,Avatar,2009,James Cameron,Twentieth Century Fox,533,237000000.0,761000000.0,Avatar,2009,"Action,Adventure,Fantasy"


In [67]:
combined_df = combined_df.drop(columns=['title_rev', 'year_rev'])

In [68]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14431 entries, tt2488496 to tt0429277
Data columns (total 8 columns):
title            14431 non-null object
year             14431 non-null object
director         14431 non-null object
production_co    14431 non-null object
rank             14431 non-null object
budget_usd       14431 non-null float64
us_gross         14431 non-null float64
genres           14317 non-null object
dtypes: float64(2), object(6)
memory usage: 1.6+ MB


In [69]:
date_df[date_df['title'] == '28 Days Later...']

,imdb_id,title,date


In [70]:
combined_df.explode('genres')

,title,year,director,production_co,rank,budget_usd,us_gross,genres
imdb_id,,,,,,,,
tt2488496,Star Wars: Episode VII - The Force Awakens,2015,J.J. Abrams,Lucasfilm,519,245000000.0,937000000.0,"Action,Adventure,Sci-Fi"
tt4154796,Avengers: Endgame,2019,Anthony Russo,Marvel Studios,111,356000000.0,858000000.0,"Action,Adventure,Drama"
tt0499549,Avatar,2009,James Cameron,Twentieth Century Fox,533,237000000.0,761000000.0,"Action,Adventure,Fantasy"
tt1825683,Black Panther,2018,Ryan Coogler,Marvel Studios,269,200000000.0,700000000.0,"Action,Adventure,Sci-Fi"
tt4154756,Avengers: Infinity War,2018,Anthony Russo,Marvel Studios,376,321000000.0,679000000.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...
tt1682940,Playback,2012,Michael A. Nickles,Fontenay Films Ltd.,"45,045",1600000.0,264.0,"Horror,Thriller"
tt4796122,Satanic,2016,Jeffrey G. Hunt,Magnet Releasing,"14,581",-1.0,252.0,Horror
tt1934452,Realms,2017,Daric Gates,DNF Pictures,"17,728",2000000.0,147.0,"Horror,Mystery"


In [ ]:
# adding in time of year next:
combined_df.set_index('title').join(date_df.set_index('title')).sample(10) 

In [ ]:
combined_df.sample(5)

In [ ]:
columns = ['title', 'year_rev', 'us_gross', 'budget_usd']
combined_df = combined_df.drop_duplicates(subset=columns)

In [ ]:
combined_df